# Sequences and Maneuvers

This notebook demonstrates targeting a frozen orbit, considering several operational constraints:
- 26 minute gap between Maneuvers
- 20 minute maximum Maneuver duration

In order to do this, we will be using QLaw as our Guiance Law. We'll also be solving the Sequence enforcing constant Local Orbital Frame direction Maneuvers and comparing that solution to the original QLaw.

In [40]:
import plotly.express as px
import pandas as pd
import numpy as np

from ostk.core.filesystem import Directory

from ostk.mathematics.geometry.d3.object import Cuboid
from ostk.mathematics.geometry.d3.object import Composite
from ostk.mathematics.geometry.d3.object import Point

from ostk.physics import Environment
from ostk.physics.coordinate import Frame
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.object.celestial import Earth
from ostk.physics.environment.object.celestial import Moon
from ostk.physics.environment.object.celestial import Sun
from ostk.physics.time import DateTime
from ostk.physics.time import Duration
from ostk.physics.time import Instant
from ostk.physics.time import Scale
from ostk.physics.time import Time
from ostk.physics.unit import Length
from ostk.physics.unit import Angle
from ostk.physics.unit import Mass

from ostk.astrodynamics import Dynamics
from ostk.astrodynamics.dynamics import Thruster
from ostk.astrodynamics.event_condition import RealCondition
from ostk.astrodynamics.event_condition import BrouwerLyddaneMeanLongCondition
from ostk.astrodynamics.event_condition import InstantCondition
from ostk.astrodynamics.event_condition import LogicalCondition
from ostk.astrodynamics.event_condition import BooleanCondition
from ostk.astrodynamics.trajectory.orbit.model.brouwerLyddaneMean import (
    BrouwerLyddaneMeanLong,
    BrouwerLyddaneMeanShort,
)
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory import Sequence
from ostk.astrodynamics.trajectory import Segment
from ostk.astrodynamics.trajectory import Propagator
from ostk.astrodynamics.trajectory import State
from ostk.astrodynamics.trajectory import StateBuilder
from ostk.astrodynamics.trajectory import LocalOrbitalFrameFactory
from ostk.astrodynamics.trajectory import LocalOrbitalFrameDirection
from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.trajectory.state import CoordinateBroker, CoordinateSubset
from ostk.astrodynamics.trajectory.state.coordinate_subset import CartesianPosition
from ostk.astrodynamics.trajectory.state.coordinate_subset import CartesianVelocity
from ostk.astrodynamics.trajectory.state.coordinate_subset import CartesianAcceleration
from ostk.astrodynamics.trajectory.orbit.model.kepler import COE
from ostk.astrodynamics.flight.system import SatelliteSystem
from ostk.astrodynamics.flight.system import PropulsionSystem
from ostk.astrodynamics.flight import Maneuver
from ostk.astrodynamics.guidance_law import QLaw
from ostk.astrodynamics.guidance_law import ConstantThrust

## User inputs

### Setup environment
- Central body: Earth
    - EGM96 20x20 gravity model
    - NRLMSISE00 Atmospheric model
- Third bodies:
    - Sun
    - Moon


In [41]:
earth = Earth.from_models(
    EarthGravitationalModel(
        EarthGravitationalModel.Type.EGM96, Directory.undefined(), 20, 20
    ),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.NRLMSISE00),
)

environment = Environment(Instant.J2000(), [earth, Sun.default(), Moon.default()])

### Setup the Satellite System, initial State and Environment
Sun Synchronous orbit:
- 580 km altitude
- LTDN of Midnight
    

In [45]:
mass = Mass.kilograms(200.0)
wet_mass = Mass.kilograms(215.0)
propulsion_system = PropulsionSystem(
    thrust_si_unit=0.0161,
    specific_impulse_si_unit=1140.26,
)

In [46]:
satellite_geometry = Composite(
    Cuboid(Point(0.0, 0.0, 0.0), np.eye(3).tolist(), [1.0, 0.0, 0.0])
)
satellite_system = SatelliteSystem(
    mass,
    satellite_geometry,
    np.eye(3),
    2.2,
    2.2,
    propulsion_system,
)

In [ ]:
state_builder = StateBuilder(
    frame=Frame.GCRF(),
    coordinate_subsets=[
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinateSubset.mass(),
        CoordinateSubset.surface_area(),
        CoordinateSubset.drag_coefficient(),
    ],
)

initial_state = state_builder.build(
    instant=Instant.parse("2025-07-15 19:59:01.000", Scale.UTC),
    coordinates=[
        -4574032.802733189,  # X
        5036207.282273548,  # Y
        1531721.604937735,  # Z
        -352.2862310871174,  # VX
        1918.952587977103,  # VY
        -7305.769392642957,  # VZ
        wet_mass.in_kilograms(),  # Mass
        satellite_system.get_cross_sectional_surface_area(),
        satellite_system.get_drag_coefficient(),
    ],
)

In [48]:
dynamics = Dynamics.from_environment(environment)
numerical_solver = NumericalSolver.default_conditional()

## Mission Planning Sequence

### Maneuvering
Target orbit as current orbit + 3 km of SMA

The optimizer will target SMA and Eccentricity.

In [51]:
initial_coe = COE.cartesian(
    (initial_state.get_position(), initial_state.get_velocity()),
    earth.get_gravitational_parameter(),
)
initial_coe

-- Classical Orbital Elements ----------------------------------------------------------------------
    Semi-major axis:                         6976350.6638760688 [m]                   
    Eccentricity:                            0.0016626581083791837                    
    Inclination:                             97.631669127535417 [deg]                 
    Right ascension of the ascending node:   310.51794467734635 [deg]                 
    Argument of periapsis:                   90.736458546498966 [deg]                 
    True anomaly:                            76.460035702561981 [deg]                 
----------------------------------------------------------------------------------------------------

In [52]:
target_coe = COE.frozen_orbit(
    semi_major_axis=Length.meters(6970.2e3),
    celestial_object=earth,
    raan=initial_coe.get_raan(),
    inclination=Angle.degrees(97.757),
)
target_coe

-- Classical Orbital Elements ----------------------------------------------------------------------
    Semi-major axis:                         6970200.0 [m]                            
    Eccentricity:                            0.0010605320050654672                    
    Inclination:                             97.757000000000005 [deg]                 
    Right ascension of the ascending node:   310.51794467734635 [deg]                 
    Argument of periapsis:                   90.0 [deg]                               
    True anomaly:                            0.0 [deg]                                
----------------------------------------------------------------------------------------------------

In [53]:
q_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLaw.Parameters(
        element_weights={
            COE.Element.SemiMajorAxis: (1.0, 50.0),
            COE.Element.Eccentricity: (1.0, 1e-4),
            COE.Element.Inclination: (1.0, 1e-1),
            COE.Element.Aop: (1.0, 1e-1),
            COE.Element.Raan: (1.0, 1e-1),
        },
        relative_effectivity_threshold=0.7,
    ),
)
thruster_dynamics = Thruster(satellite_system, q_law)

#### Maneuver segment Event conditions

In [54]:
semi_major_axis_condition = BrouwerLyddaneMeanLongCondition.semi_major_axis(
    criterion=RealCondition.Criterion.StrictlyNegative,
    frame=Frame.GCRF(),
    semi_major_axis=RealCondition.Target(target_coe.get_semi_major_axis()),
    gravitational_parameter=earth.get_gravitational_parameter(),
)

eccentricity_condition = BrouwerLyddaneMeanLongCondition.eccentricity(
    criterion=RealCondition.Criterion.StrictlyNegative,
    frame=Frame.GCRF(),
    eccentricity=RealCondition.Target(target_coe.get_eccentricity()),
    gravitational_parameter=earth.get_gravitational_parameter(),
)

inclination_condition = BrouwerLyddaneMeanLongCondition.inclination(
    frame=Frame.GCRF(),
    target_range=[
        target_coe.get_inclination() - Angle.degrees(1e-2),
        target_coe.get_inclination() + Angle.degrees(1e-2),
    ],
    gravitational_parameter=earth.get_gravitational_parameter(),
)

raan_condition = BrouwerLyddaneMeanLongCondition.raan(
    frame=Frame.GCRF(),
    target_range=[
        target_coe.get_raan() - Angle.degrees(1e-2),
        target_coe.get_raan() + Angle.degrees(1e-2),
    ],
    gravitational_parameter=earth.get_gravitational_parameter(),
)

aop_condition = BrouwerLyddaneMeanLongCondition.aop(
    frame=Frame.GCRF(),
    target_range=[
        target_coe.get_aop() - Angle.degrees(1e-2),
        target_coe.get_aop() + Angle.degrees(1e-2),
    ],
    gravitational_parameter=earth.get_gravitational_parameter(),
)

orbit_condition = LogicalCondition(
    "orbit condition",
    LogicalCondition.Type.And,
    [
        semi_major_axis_condition,
        eccentricity_condition,
        inclination_condition,
        raan_condition,
        aop_condition,
    ],
)

In [55]:
duration_condition = RealCondition.duration_condition(
    criterion=RealCondition.Criterion.StrictlyPositive,
    duration=Duration.minutes(20.0),
)

#### Maneuver Segment

In [56]:
# Stop if either condition is met
maneuver_condition = LogicalCondition(
    "maneuver condition",
    LogicalCondition.Type.Or,
    [orbit_condition, duration_condition],
)

maneuver_segment = Segment.maneuver(
    name="Maneuver Segment",
    event_condition=maneuver_condition,
    thruster_dynamics=thruster_dynamics,
    dynamics=dynamics,
    numerical_solver=numerical_solver,
)

constant_lof_maneuver_segment = Segment.constant_local_orbital_frame_direction_maneuver(
    name="Constant LOF Maneuver Segment",
    event_condition=maneuver_condition,
    thruster_dynamics=thruster_dynamics,
    dynamics=dynamics,
    numerical_solver=numerical_solver,
    local_orbital_frame_factory=LocalOrbitalFrameFactory.VNC(Frame.GCRF()),
)

### Coasting

In [57]:
# coast for atleast 26.0 minutes
duration_condition = RealCondition.duration_condition(
    criterion=RealCondition.Criterion.StrictlyPositive,
    duration=Duration.minutes(26.0),
)

coast_segment = Segment.coast(
    name="Maneuver Segment",
    event_condition=duration_condition,
    dynamics=dynamics,
    numerical_solver=numerical_solver,
)

### Execute

In [ ]:
maximum_propagation_duration = Duration.days(7.0)

# Satellite using QLaw
sequence_1 = Sequence(
    segments=[coast_segment, maneuver_segment],
    dynamics=dynamics,
    numerical_solver=numerical_solver,
    maximum_propagation_duration=maximum_propagation_duration,
)

# Satellite using QLaw but enforcing constant LoF direction Maneuvers
sequence_2 = Sequence(
    segments=[coast_segment, constant_lof_maneuver_segment],
    dynamics=dynamics,
    numerical_solver=numerical_solver,
    maximum_propagation_duration=maximum_propagation_duration,
)

In [17]:
repetition_count = 200

solution_1 = sequence_1.solve(
    state=initial_state,
    repetition_count=repetition_count,
)

solution_2 = sequence_2.solve(
    state=initial_state,
    repetition_count=repetition_count,
)

In [18]:
states_1 = solution_1.get_states()
states_2 = solution_2.get_states()

In [20]:
maneuvers_1 = [
    maneuver
    for segment_solution in solution_1.segment_solutions
    for maneuver in segment_solution.extract_maneuvers(Frame.GCRF())
]

maneuvers_2 = [
    maneuver
    for segment_solution in solution_2.segment_solutions
    for maneuver in segment_solution.extract_maneuvers(Frame.GCRF())
]

In [21]:
print(len(maneuvers_1))
print(len(maneuvers_2))

182
183


In [ ]:
def populate_data_frame(
    states: list[State],
    maneuvers: list[Maneuver],
    source: str,
    state_gap: Duration = Duration.minutes(10.0),
):
    data = []
    last_instant = None

    for state in states:
        if last_instant is None or (state.get_instant() - last_instant) > state_gap:
            blm = BrouwerLyddaneMeanLong.cartesian(
                (state.get_position(), state.get_velocity()),
                earth.get_gravitational_parameter(),
            )

            in_maneuver = False
            for maneuver in maneuvers:
                if maneuver.get_interval().contains_instant(state.get_instant()):
                    in_maneuver = True
                    break

            sma = blm.get_semi_major_axis().in_kilometers()
            data.append(
                {
                    "altitude": float(
                        sma - earth.get_equatorial_radius().in_kilometers()
                    ),
                    "mass": float(state.get_coordinates()[6]),
                    "sma": float(sma),
                    "eccentricity": float(blm.get_eccentricity()),
                    "inclination": float(blm.get_inclination().in_degrees()),
                    "periapsis": float(blm.get_periapsis_radius().in_kilometers()),
                    "apoapsis": float(blm.get_apoapsis_radius().in_kilometers()),
                    "raan": float(blm.get_raan().in_degrees()),
                    "aop": float(blm.get_aop().in_degrees()),
                    "mean_anomaly": float(blm.get_mean_anomaly().in_degrees(0.0, 360.0)),
                    "true_anomaly": float(blm.get_true_anomaly().in_degrees(0.0, 360.0)),
                    "argument_of_latitude": float(
                        blm.get_argument_of_latitude().in_degrees(0.0, 360.0)
                    ),
                    "time": state.get_instant().get_date_time(Scale.UTC),
                    "x": state.get_coordinates()[0],
                    "y": state.get_coordinates()[1],
                    "z": state.get_coordinates()[2],
                    "r": np.linalg.norm(state.get_coordinates()[:3]),
                    "in_maneuver": in_maneuver,
                    "phase": f"{source} - {in_maneuver}",
                    "source": source,
                }
            )
            last_instant = state.get_instant()

    return pd.DataFrame(data)

In [23]:
df_1 = populate_data_frame(
    states=states_1,
    maneuvers=maneuvers_1,
    source="QLaw",
)

df_2 = populate_data_frame(
    states=states_2,
    maneuvers=maneuvers_2,
    source="Constant LOF QLaw",
)

combined_df = pd.concat([df_1, df_2])

In [24]:
figure_settings = {
    "height": 600,
    # "width": 1200,
    "symbol": "source",
    "color": "source",
    "color_discrete_map": {
        "QLaw": "grey",
        "Constant LOF QLaw": "orange",
        "QLaw - False": "grey",
        "QLaw - True": "black",
        "Constant LOF QLaw - False": "orange",
        "Constant LOF QLaw - True": "red",
    },
}

In [25]:
figure = px.scatter(
    combined_df,
    x="time",
    y="sma",
    title="SMA [km]",
    **figure_settings,
)

figure.show()

In [26]:
figure = px.scatter(
    combined_df,
    x="time",
    y="eccentricity",
    title="Eccentricity",
    **figure_settings,
)

figure.show()

In [27]:
figure = px.scatter(
    combined_df,
    x="time",
    y="inclination",
    title="Inclination",
    **figure_settings,
)

figure.show()

In [30]:
figure = px.scatter(
    combined_df,
    x="time",
    y="aop",
    title="Argument of Perigee",
    **figure_settings,
)

figure.show()

In [31]:
figure = px.scatter(
    combined_df,
    x="time",
    y="raan",
    title="RAAN",
    **figure_settings,
)

figure.show()

---